# 간단한 텍스트 전처리 파이프라인 구축

>범주, 시간, 작성자, 기타 속성등의 메타데이터를 분석하면 말뭉치에 대한 첫 통찰을 얻는 것으로 실제 콘텐츠에선 깊이 들어갈 수록 하위 집합 혹은 특정 기간에 자주 사용된 단어를 탐색하는 것이 흥미진진합니다.

<br>

**이번엔 텍스트를 준비하기 위한 단계로 한 작업의 출력이 다음 작업의 입력을 형성하므로 원본 텍스트를 여러 토큰으로 변환하는 processing pipeline 설정한다.**





## 전처리 파이프라인

- 텍스트 데이터

- 변환(대문자를 소문자로 변환)

- 토큰화(정규표현식 활용)

- 불용어 제거

- 준비된 토큰

- 임의의 불용어 목록 만들기

In [2]:
# 필요한 라이브러리 및 데이터 로드
import pandas as pd
import numpy as np

un = pd.read_csv("/content/drive/MyDrive/UN/un-general-debates.csv")

### 정규 표현식을 이용한 토큰화

**토큰화**

>일련의 문자에서 단어를 추출하는 절차로 서양의 단어는 공백과 구두점 문자로 구분하므로, 가장 간단하고 빠른 토큰화 함수는 공백으로 분할하는 파이썬의 기본 str.split()메서드인데, 더 유연한 것은 정규 표현식을 사용한다.

<br>

정규 표현식과 파이썬 라이브러리인 re와 regex를 활용하여 단어를 일치시키는 간단한 패턴을 적용하고 하나 이상의 문자와 숫자, 하이픈으로 구성된 문자열을 단어로 정의한다.

<br>

순수한 숫자만 들어 있는 문자열은 보통 말뭉치의 나짜나 연설 또는 세션 식별자를 나타내므로 건너뛴다

In [6]:
"""
자주 사용되는 표현식 [A-Za-z]는 특수 문자가 붙은 문자는
놓치므로 문자 일치에 좋은 옵션이 아니다.

훨씬 더 나은 방식은  유니코드 문자를 선택하는 POSIX 문자 클래스\p{L}다.
POSIX 문자 클래스로 작업하려면 re 대신 regex 라이브러리가 필요하다.
"""

import regex as re

def tokensize(text):
    return re.findall(r'[\w-]*\p{L}[\w-]*',text)

text = "Let's defeat SARS-CoV-2 together in 2020!"
tokens = tokensize(text)
print(":".join(tokens))

Let:s:defeat:SARS-CoV-2:together:in


### 불용어 처리

**불용어**

>텍스트에서 가장 자주 사용되는 단어는 한정사, 조동사, 대명사, 부사 같은 일반적인 단어이다.

<br>

불용어는 많은 정보를 전달하지 않지만 출현 빈도가 높기 때문에 중요한 내용을 놓치게 만들지만 데이터 분석이나 모델 훈련 전에는 불용어를 제거하는 경우가 많다.

<br>

미리 정의된 불용어 목록을 이용해서 불용어를 제거하는 방법으로 소개하고, 모든 언어는 일반적인 불용어 목록이 있고, 이 목록은 거의 모든 NLP 라이브러리에 통합되므로 NLTK의 불용어 목록을 사용하지만 사용자 지정 단어 목록을 필터로 사용할 수 있다.

<br>

**빠른 조회를 위해 목록을 항상 파이썬의 Set 타입으로 변환하고 해시 기반 구조로 조회 시간이 거의 일정하다.**

In [ ]:
!pip install nltk
from nltk.corpus import stopwords
nltk.download("all")

In [ ]:
nltk.download("all")

In [14]:
import nltk
stopwords = set(nltk.corpus.stopwords.words('english'))

In [15]:
"""
리스트 컴프리헨션을 사용하여 구현한 함수로 불용어를 제거한다.
NLTK목록에는 소문자 단어만 포함되므로 입력 토큰을 소문자로 변환한다.
"""

def remove_stop(tokens):
    return [t for t in tokens if t.lower() not in stopwords]

"""
도메인에 따라 주로 사용되는 불용어를 미리 정의된 목록에 추가할 수 있다.
예를 들어, 이메일 분석 시 dear or regards라는 단어가 문서마다 등장한다.

이와 달리 일부 단어를 미리 정의된 목록에 추가하여 불용어에서 제외할 수도 있다.
목록에서 불용어를 추가하거나 삭제할 땐 집합 연산자 & 및 -를 사용한다.
"""
include_stopwords = {'dear',"regards","must","would","also"}
exclude_stopwords = {"against"}


stopwords -= exclude_stopwords#삭제
stopwords |= include_stopwords#추가


NLTK의 불용어 목록은 보수적으로 179개 단어만 포함되고, 놀랍게도 will은 불용어 아니지만 wouldn't는 불용어로 간주된다.

<br>


이처럼 미리 정의된 불용어 목록의 고질적 문제인 불일치의 한 예이고, 불용어를 제거하는 것이 위험한 이유는 불용어를 제거하면 의미론적 분석 성능에 영향을 미친다.

---

### 불용어를 제거하는 것이 위험한 이유

**불용어 제거는 대충 만든 규칙으로 기반하고, 불용어 목록을 유심히 살펴서 소중한 정보가 삭제되지 않게 해야 한다.**

<br>

>예: I don't like ice cream

NLTK와 스페이시 불용어 목록에는 I와 don't가 모두 있고, 이 불용어를 빼면 like와 ice cream민 남기에 이런 식의 전처리는 모든 종류의 감성 분석을 크게 왜곡한다.

<br>

>TF-IDF 가중치는 자주 발생하는 단어의 가중치를 자동으로 낮추지만 해당 용어는 어휘 목록에 유지한다.

<br>

#### 정리

물론, 미리 정의된 목록에 불용어를 추가하거나 사용자가 지정한 불용어 목록을 사용하지 않고, 대신 문서에 등장한 단어 중 빈도가 80%이상인 단어를 불용어로 처리하는 편이 유용할 수 있다.

왜냐하면, 이런 빈도 높은 단어는 내용 파악을 어렵게 하므로 사이킷런의 벡터화 객체에서 사용하는 매개변수 max_df가 정확히 이런 단어를 제거한다.

또 다른 방법은 단어를 단어 유형(품사)에 따라서 필터링하는 것이다.

### 코드 한 줄로 파이프라인 처리

>말 뭉치 문서가 포함된 데이터에서 각 단어를 소문자화하고, 불용어가 제거된 토큰화된 텍스트가 포함된 tokens라는 새 열을 만들고, 이를 위해선 processing pipeline에 확장 가능한 패턴(extensible pattern)을 사용한다.



In [17]:
"""
1. 모든 텍스트를 소문자로 변경
2. 토큰화
3. 불용어 제거
위의 세 가지 방향을 파이프라인을 확장해서 다른 작업 추가
"""
pipeline = [str.lower,tokensize,remove_stop]


"""
아래의 함수 내용을 넣으면 pandas의 map, apply연산의 완벽한 사용 사례
다른 함수를 매개변수로 사용하는 map 및 apply같은 함수라고도 한다
"""
def prepare(text, pipeline):
    tokens = text
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

**판다스의 고차 함수**

*파이썬의 pandarallel패키지는 병렬 버전의 map 및 apply를 제공한다.*

|함수|설명|
|------|---|
|Series.map|Pandas Series에서 요서별로 작동|
|Series.apply|맵과 동일하지만 추가 매개변수 허용|
|DataFrame.applymap|Pandas DataFrame의 요소별 요소(Series의 맵과 동일|
|DataFrame.apply|데이터프레임의 행 또는 열에서 작동하고 집계 지원|

In [18]:
#판다스의 고차 연산을 사용하면 코드 한 줄로 다양한 함수를  사용한 데이터 변환 가능

un['tokens'] = un['text'].apply(prepare,pipeline=pipeline)

토큰 열은 각 문서에서 추출된 토큰을 포함한 파이썬 리스트가 되고, 추가 열은 데이터프레임의 메모리 사용량을 두 배로 늘리지만 추가 분석을 위해서 토큰에 직접 빠르게 엑세스할 수 있다.

<br>

분석 중에 즉시 토큰화가 수행되면 성능이 메모리 소비를 상쇄하여 분석 전에 한 번 토큰화하고 메모리를 소비하거나 아니면 즉석에서 토큰화하고 기다린다.

In [19]:
#토큰 목록의 길이를 포함한 열을 추가한다.
un['num_tokens'] = un['tokens'].map(len)

In [ ]:
un

>tqdm은 파이썬에서 작업 진행률 표시줄을 위한 훌륭한 라이브러리로 범위 대신 tqdm_range를 사용하여 기존 반복문을 지원하고 데이터프레임에 progress_map 및 progress_apply 작업을 제공하여 판다스 지원한다.